In [ ]:
from rockit import *
# from casadi import SX, MX, vertcat, cross, fabs, norm_2
import casadi as ca
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from pyquaternion import Quaternion

In [ ]:
ocp = Ocp(t0=0, T=FreeTime(100))

In [ ]:
Tmax = 0.000953
Wmax = 1570.79632679
Vmax = 6
a_1 = np.array([[1],[0],[0]])
a_2 = np.array([[0],[1],[0]])
a_3 = np.array([[0],[0],[1]])
a_mat = np.concatenate((a_1, a_2, a_3), axis=1)
I = np.array([[0.04, 0.00, 0.00],
              [0.00, 0.05, 0.02],
              [0.00, 0.02, 0.05]])
I_inv = np.linalg.inv(I)
I_rw = 6.25E-05
I_wheels = I_rw*(a_1*np.transpose(a_1) + a_2*np.transpose(a_2) + a_3*np.transpose(a_3))

In [ ]:
q = ocp.state(4)
w = ocp.state(3)
Omega = ocp.state(3)
V = ocp.control(3)
# Q = ocp.algebraic(4, 4)
# Q[0,0] = 0
# Q[0,1] = -w[0]
# Q[0,2] = -w[1]
# Q[0,3] = -w[2]

# Q[1,0] = w[0]
# Q[1,1] = 0
# Q[1,2] = -w[2]
# Q[1,3] = w[1]

# Q[2,0] = w[1]
# Q[2,1] = w[2]
# Q[2,2] = 0
# Q[2,3] = w[0]

# Q[3,0] = w[2]
# Q[3,1] = -w[1]
# Q[3,2] = w[0]
# Q[3,3] = 0

In [ ]:
# Assign derivatives

ocp.set_der(q, ca.vertcat(
    -w[0]*q[1]-w[1]*q[2]-w[2]*q[3],
    w[0]*q[0] +w[1]*q[3]-w[2]*q[2],
    -w[0]*q[3]+w[1]*q[0]+w[2]*q[1],
    w[0]*q[2] -w[1]*q[1]+w[2]*q[0]
))
# ocp.set_der(q[0], -w[0]*q[1]-w[1]*q[2]-w[2]*q[3])
ocp.set_der(w, -(Tmax/Vmax)*a_mat@V + (Tmax/Wmax)*a_mat@Omega - ca.cross(w, (I@w) + (I_wheels@w) + I_rw*(a_mat@Omega)))
ocp.set_der(Omega, (Tmax/(Vmax*I_rw))*V - (Tmax/(Wmax*I_rw))*Omega - a_mat.T@w)

In [ ]:
# Constraints

ocp.subject_to(q[0]**2 + q[1]**2 + q[2]**2 + q[3]**2 == 1)
ocp.subject_to(-6 <= (V[0] <= 6))
ocp.subject_to(-6 <= (V[1] <= 6))
ocp.subject_to(-6 <= (V[2] <= 6))

In [ ]:
# Boundary conditions

ocp.subject_to(ocp.at_t0(q) == np.array([1,0.0,0.0,0.0]))
# ocp.subject_to(ocp.at_tf(q) == np.array([-0.0434363, -0.5855824, -0.7539713, -0.2945059]))
ocp.subject_to(ca.norm_2(ocp.at_tf(q) - np.array([-0.0434363, -0.5855824, -0.7539713, -0.2945059])) <= 1e-5)

# ocp.subject_to(ocp.at_tf(q) == np.array([-4.34362998e-02, -5.85582404e-01, -7.53971357e-01, -2.94505902e-01]))

ocp.subject_to(ocp.at_t0(w) == np.array([0,0.0,0.0]))
ocp.subject_to(ocp.at_tf(w) == np.array([0,0.0,0.0]))

ocp.subject_to(ocp.at_t0(Omega) == np.array([0,0.0,0.0]))
ocp.subject_to(ocp.at_tf(Omega) == np.array([0,0.0,0.0]))

In [ ]:
ocp.add_objective(ocp.T)

In [ ]:
# ocp.solver('ipopt', {"ipopt": {"max_iter": 3000}})
ocp.solver('ipopt', {"ipopt": {"max_iter": 500, "mu_init": 1e-3}})

method = SingleShooting(N=200, intg='rk')
ocp.method(method)

In [ ]:
sol = ocp.solve()

In [ ]:
sol = ocp.non_converged_solution

In [ ]:
%matplotlib inline

ocp.spy()

In [ ]:
tsa, qa = sol.sample(q, grid='control')
# tsb, x1b = sol.sample(x1, grid='integrator')
# tsc, x1c = sol.sample(x1, grid='integrator', refine=100)
# plot(tsa, x1a, '-')
# plot(tsb, x1b, 'o')
# plot(tsc, x1c, '.')
qa[-1] # -4.34362998e-02, -5.85582404e-01, -7.53971357e-01, -2.94505902e-01

In [ ]:
%matplotlib inline
plt.title('Quaternion vector')
plt.plot(tsa, qa[:,0], 'r', label='q1')
plt.plot(tsa, qa[:,1], 'g', label='q2')
plt.plot(tsa, qa[:,2], 'b', label='q3')
plt.plot(tsa, qa[:,3], 'tab:orange', label='q4')
plt.legend()
plt.show()

In [ ]:
tsb, Vb = sol.sample(V, grid='control')

In [ ]:
%matplotlib inline
plt.title("Reaction wheel voltages")
plt.plot(tsb, Vb[:,0], 'r', label="Wheel 1")
plt.plot(tsb, Vb[:,1], 'g', label="Wheel 2")
plt.plot(tsb, Vb[:,2], 'b', label="Wheel 1")
plt.legend()
plt.show()

In [ ]:
tsc, wc = sol.sample(w, grid='control')

In [ ]:
%matplotlib notebook
plt.plot(tsc, wc[:,0], 'r')
plt.plot(tsc, wc[:,1], 'g')
plt.plot(tsc, wc[:,2], 'b')
plt.show()

In [ ]:
tsd, Omegad = sol.sample(Omega, grid='control')

In [ ]:
%matplotlib inline
plt.title("Reaction wheel velocities (rad/s)")
plt.plot(tsd, Omegad[:,0], 'r', label="Wheel 1")
plt.plot(tsd, Omegad[:,1], 'g', label="Wheel 2")
plt.plot(tsd, Omegad[:,2], 'b', label="Wheel 3")
plt.legend()
plt.show()

In [ ]:
quat_desired = Quaternion(np.array([-0.0434363, -0.5855824, -0.7539713, -0.2945059]))

In [ ]:
err = []
for quat in qa:
    quat_cur = Quaternion(quat)
    err.append(2*np.arccos((quat_cur.conjugate*quat_desired).real)*180/np.pi)

In [ ]:
%matplotlib inline
plt.plot(tsa, err)
plt.title("Euler error (deg)")
plt.plot()